## 神经网络的内部原理

---

#### 介绍

为什么神经网络能够有如此高的识别率，学习前与学习后的神经网络到底有什么不同？本实验将逆其道而行之，将输出的标签作为输入，将图片作为输出，观察将一个 0 的标签输入模型中，模型会输出怎样的图片。进而了解神经网络的内部原理。

#### 知识点

- 神经网络的黑盒子
- 神经网络的逆运算
- 神经网络的向后查询
- 扩大数据集的方法

---

### 神经网络大脑内部

在求解各种各样无法使用简约明快的规则解决的问题时，神经网络发挥了重要作用。想象一下，写下一组规则，将这些规则应用于手写数字图像，来确定数字是什么，这件事并不是那么容易，并且我们的尝试也可能不会那么成功。 

#### 神秘的黑盒子 

一旦神经网络得到了训练，并且在测试数据上表现良好，那么基本上你就拥有了一个神秘的黑盒子。你不知道这个黑盒子如何计算出答案，但是它确实成功地计算出了答案。

如果你只对答案感兴趣，而不真正关心它们如何得出这个答案的，那么对你来说，这就不是一个问题了。但是，我要指出这是这些机器学习方法类型的缺点，即虽然黑盒子（神经网络）已经学会如何求解问题，但是其所学习到的知识常常不能转化为对问题的理解和智慧。

让我们来看看是否可以到神经网络内部一探究竟，是否能够理解神经网络所学习到的知识，将神经网络通过训练搜集到的知识可视化。 

我们可以观察权重，这毕竟是神经网络学习的内容。但是，权重不太可能告诉我们太多信息。特别是，神经网络的工作方式是将学习分布到不同的链接权重中。这种方式使得神经网络对损坏具有了弹性，这就像是生物大脑的运行方式。删除一个节点甚至相当多的节点，都不太可能彻底破坏神经网络良好的工作能力。

这里有一个疯狂的想法。

#### 向后查询 

在通常情况下，我们馈送给已受训练的神经网络一个问题，神经网络弹出一个答案。在我们的例子中，这个问题是人类的手写数字图像。答案是表示数字 0 到 9 中的某个标签。

如果将这种方式反转，向后操作，会发生什么呢？如果馈送一个标签到输出节点，通过已受训练的网络反向输入信号，直到输入节点弹出一个图像，那会怎么样？下图显示了正常的正向查询和疯狂的反向向后查询的想法。

<img width="400px" src="https://doc.shiyanlou.com/courses/uid1166617-20191228-1577514947479/wm">

我们已经知道如何通过网络传播信号，使用链接权重调节信号，在应用激活函数之前在节点处重新组合信号。除了使用的是逆激活函数以外，所有这一切操作也都适用于反向传播信号。如果 $y = f(x)$ 是正向激活函数，那么这个函数的逆就是 $x = g(y)$。使用简单的代数，求出逻辑函数的逆，也并非难事：

$$y = \frac{1}  {(1 + e^{-x})}$$

$$1 + e^{-x} = \frac{1}{y}$$

$$ e^{-x} = \frac{1}{y} -1 = \frac{1 - y }{ y}$$ 

$$ -x = ln( \frac{1 - y }{ y}) $$

$$x = ln (\frac{ y }{ 1 -y} ) $$

这就是所谓的对数函数，就像 Python 为逻辑 S 函数提供 scipy.special.expit() 一样，Python 中的 scipy.special 库也提供了这个函数，即 scipy.special.logit()。

在应用逆激活函数 logit() 之前，我们需要确保信号是有效的。这是什么意思呢？还记得吧，逻辑 S 函数接受了任何数值，输出 0 和 1 之间的某个值，但是不包括 0 和 1 本身。逆函数必须接受相同的范围0和1之间的某个值，不包括 0 和 1，弹出任何正值或负值。为了实现这一目标，我们简单地接受输出层中的所有值，应用 logit()，并将它们重新调整到有效范围。我选择的范围为 0.01 至0.99。 

因此，我们为原来的神经网络类添加一个反向查询函数 backquery()

In [ ]:
import numpy
import scipy.special
import matplotlib.pyplot
%matplotlib inline


# 神经网络类的定义，由于初始化、训练以及查询已在之前的章节中讲解，这里不做介绍
# 重点介绍反向查询函数 backquery（）
class neuralNetwork:
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        self.wih = numpy.random.normal(0.0, pow(self.inodes, -0.5),
                                       (self.hnodes, self.inodes))
        self.who = numpy.random.normal(0.0, pow(self.hnodes, -0.5),
                                       (self.onodes, self.hnodes))
        self.lr = learningrate
        self.activation_function = lambda x: scipy.special.expit(x)
        self.inverse_activation_function = lambda x: scipy.special.logit(x)
        pass

    # 神经网络的训练函数
    def train(self, inputs_list, targets_list):
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T
        hidden_inputs = numpy.dot(self.wih, inputs)
        hidden_outputs = self.activation_function(hidden_inputs)
        final_inputs = numpy.dot(self.who, hidden_outputs)
        final_outputs = self.activation_function(final_inputs)
        output_errors = targets - final_outputs
        hidden_errors = numpy.dot(self.who.T, output_errors)
        self.who += self.lr * numpy.dot((output_errors * final_outputs *
                                         (1.0 - final_outputs)),
                                        numpy.transpose(hidden_outputs))
        self.wih += self.lr * numpy.dot((hidden_errors * hidden_outputs *
                                         (1.0 - hidden_outputs)),
                                        numpy.transpose(inputs))
        pass

    def query(self, inputs_list):
        inputs = numpy.array(inputs_list, ndmin=2).T
        hidden_inputs = numpy.dot(self.wih, inputs)
        hidden_outputs = self.activation_function(hidden_inputs)
        final_inputs = numpy.dot(self.who, hidden_outputs)
        final_outputs = self.activation_function(final_inputs)
        return final_outputs
    def backquery(self, targets_list):
        # 将标签转换为神经网络认识的数组形式
        final_outputs = numpy.array(targets_list, ndmin=2).T

        # 反向计算输出层的激活函数
        final_inputs = self.inverse_activation_function(final_outputs)

        # 反向计算隐藏层的输出
        hidden_outputs = numpy.dot(self.who.T, final_inputs)
        # scale them back to 0.01 to .99
        hidden_outputs -= numpy.min(hidden_outputs)
        hidden_outputs /= numpy.max(hidden_outputs)
        hidden_outputs *= 0.98
        hidden_outputs += 0.01

        # 反向计算隐藏层的激活函数
        hidden_inputs = self.inverse_activation_function(hidden_outputs)

        # 反向计算出隐藏层的输入
        inputs = numpy.dot(self.wih.T, hidden_inputs)
        # 将缩小的值，反向扩大
        inputs -= numpy.min(inputs)
        inputs /= numpy.max(inputs)
        inputs *= 0.98
        inputs += 0.01

        return inputs

按照上一实验的操作，我们加载数据，并且模型进行训练。

In [ ]:
!wget -nc https://labfile.oss.aliyuncs.com/courses/1500/minist-data.zip
!unzip -o "minist-data.zip" 

In [ ]:
#初始化神经网络
input_nodes = 784 #输入层节点选择 784 个的原因是因为一张手写数字图像的大小为 28*28=784
hidden_nodes = 100 
output_nodes = 10 #最后的答案是 0-9 十个数字
learning_rate = 0.1

# 创建一个神经网络实例
n = neuralNetwork(input_nodes,hidden_nodes,output_nodes, learning_rate)

#训练神经网络
# 加载训练数据，此时用的是，全部的数据
training_data_file = open("mnist_train.csv", 'r')
#其他代码，同上
training_data_list = training_data_file.readlines()
training_data_file.close() 

print("开始训练，请耐心等待（3~5 min）....")
for record in training_data_list:
    all_values = record.split(',')
    inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
    targets = numpy.zeros(output_nodes) + 0.01
    targets[int(all_values[0])] = 0.99
    n.train(inputs, targets)
    pass
print("训练完毕")

### 标签 “0”

如果我们使用标签“0”进行反向查询，会发生什么情况。也就是说，我们向输出节点展示了一些值，除了使用值 0.99 展示给第一个节点表示标签“0”，其余节点都展示了 0.01。换句话说，也就是数组 [0.99, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,0.01] 。

In [ ]:
#这里将标签 0 进行反向传播
label = 0
#创造一个标签
targets = numpy.zeros(output_nodes) + 0.01
targets[label] = 0.99
print(targets)

# 将标签传入神经网络中，进行反向查询
image_data = n.backquery(targets)
# 查看反向传出来的图片
matplotlib.pyplot.imshow(image_data.reshape(28,28), cmap='Greys', interpolation='None')

上图显示了输入节点弹出的图像。

这真是太有趣了！

这个图像让我们对神经网络的“大脑”有了一种深刻的见解。这个图像是什么意思？该如何解释这个图像呢？

我们注意到最主要的特征是，图像中的圆形。我们是在询问神经网络——对于答案 “0”，最理想的问题是什么，因此，这个图像是有道理的。

我们也注意到深色、浅色和一些介中的灰色区域。

- 深色区域是问题图像中应该使用笔来标记的部分，这些部分看起来组成了 0 的形状轮廓。 
- 浅色区域是问题图像中应该没有任何笔痕的部分，这些部分形成了0形状的中间部分。 
- 大体上，神经网络对灰色区域不是很敏感。 

因此，粗略来讲，我们实际上已经理解了，针对如何将图像归类为标签 “0”，神经网络已经学习到的知识。 

这是一种难得的见解，对于较多层、较复杂的神经网络或较复杂的问题而言，可能没有如此容易解释的结果。我们鼓励你进行实验，亲自动手试一试。

### 更多的大脑扫描 

下图显示了其他数字向后查询的结果。

<img width="400px" src="https://doc.shiyanlou.com/courses/uid1166617-20191228-1577528443026/wm">

哇！同样是一些非常有趣的图像，就像使用超声波扫描神经网络的大脑一样。

关于这些图像，我们做了一些注解：

-  “7” 真的很清楚。可以看到在查询图像中标记的深色位置，强烈暗示了这是标签 “7” 。也可以看到额外的“白色”区域，这些区域没有任何标记。这两个特点结合起来，指示出了这是 “7” 。 
- 这同样适用于数字 “3” ，有标记的深色区域指示出了 “3” ，白色的区域也非常清晰。 “2” 和 “5” 具有类似的清晰度。 
- 数字“4”有点有趣，这个形状出现在 4 个象限中，是 4 个互相分隔的区域。 
- “8”主要是由“雪人”构成的，这个“雪人”由白色区域形成，表明 8 的特征在于保持了“头部和身体”区的标记。 
-  数字 “1” 令人相当费解。这看起来好像神经网络较多关注无需标记的区域，而较少关注需要标记的区域。没关系，这就是网络从样本中学到的知识。 
- 数字 “9” 一点都不清楚。它有一个明确的深色区域，还有一些形状相对精细的白色区域。这就是神经网络所学习到的知识，总体来说，当与网络学会的其他数字结合时，这允许神经网络的表现达到了 97.5% 的准确度。我们观察一下这个图片，并得出结论，有更多的培训样本将有助于神经网络学到更清晰的 “9” 的模板。 

现在，你对神经网络大脑的工作方式应该有了一个深刻的了解了吧。

### 创建新的训练数据：旋转图像

如果思考一下 MNIST 训练数据，你就会意识到，这是关于人们所书写数字的一个相当丰富的样本集。这里有各种各样、各种风格的书写，有的写得很好，有的写得很糟。 

神经网络必须尽可能多地学习这些变化类型。在这里，有多种形式的数字 “4” ，有些被压扁了，有些比较宽，有些进行了旋转，有些顶部是开放的，有些顶部是闭合的，这对神经网络的学习都是有帮助的。

如果我们能够创造更多的变化类型作为样本，会不会有用处呢？如何做到这一点呢？再多收集几千个人类手写数字样本，对我们来说有点不太容易。我们可以这样做，但是工作量有点大。 

一个很酷的想法就是利用已有的样本，通过顺时针或逆时针旋转它们，比如说旋转 10 度，创建新的样本。对于每一个训练样本而言，我们能够生成两个额外的样本。我们可以使用不同的旋转角度创建更多的样本，但是，目前，让我们只尝试 +10 和 -10 两个角度，看看这种想法能不能成功。

同样，Python 的许多扩展包和程序库都很有用。ndimage.interpolation.rotate() 可以将数组转过一个给定的角度，这正是我们所需要的。请记住，由于我们将神经网络设计成为接收一长串输入信号，因此输入的是 784 长的一串一维数字。我们需要将这一长串数字重新变成 $28\times 28$ 的数组，这样就可以旋转这个数组。然后在将这个数组馈送到神经网络之前，将数组解开，重新变成一长串的 784 个信号。

假设得到了先前的 scaled_input 数组，下列代码演示了如何使用 ndimage.interpolation.rotate() 函数：

In [ ]:
import scipy.ndimage
#获得数据集中的第一条数据，并把它转换为一张图片
all_values = training_data_list[0].split(',')
image_array = numpy.asfarray(all_values[1:]).reshape((28, 28))

# 对原图进行旋转
# 将原图逆时针旋转10°
inputs_plus10_img = scipy.ndimage.interpolation.rotate(image_array ,
                                                       10 ,cval=0.01 ,
                                                       reshape=False)
# 将原图顺时针旋转10°
inputs_minus10_img =scipy.ndimage.interpolation.rotate( image_array,
                                                       -10 ,cval=0.01 ,
                                                       reshape=False)

#显示变化后的这两张图片
matplotlib.pyplot.imshow(inputs_plus10_img, cmap='Greys', interpolation='None')
matplotlib.pyplot.figure()
matplotlib.pyplot.imshow(inputs_minus10_img, cmap='Greys', interpolation='None')

可以清楚地看到这种方式的好处。原始图像的版本旋转 +10 度，提供了一个样本，就像某些人的书写风格是将 5 向后倾斜。将原来图片的版本顺时针旋转 10 度更有趣。和原始的版本相比，这个版本在某种意义上是更具代表性的学习图片。

设定学习率为 0.1 ，并且只使用一个训练世代，初始运行神经网络，所得的性能是 0.9669 。这对于没有使用额外旋转图像进行训练的神经网络的性能 0.954 而言，是一个很大的进步。这样的表现，和列在 Yann LeCun 网站中的记录相比也已经是名列前茅了。 

让我们进行一系列的实验，改变世代的数目看看是否能够让已经不错的表现更上一层楼。现在，我们创建了更多的训练数据，可以采用更小、更谨慎的学习步长，因此将学习率减少到 0.01 ，这样就总体上延长了学习时间。

请记住，由于特定的神经网络架构或训练数据的完整性，事情很可能存在内在的限制，因此我们不会期待得到 98% 或以上的准确度，或者甚至是 100% 的准确度。我们说“特定的神经网络架构”，意思是每一层节点数目的选择、隐藏层的选择和激活函数的选择等。

我们旋转训练图像的角度，将其作为额外的训练样本，下图显示了在这种情况下的神经网络的性能。同时，下图也显示了没有使用额外旋转的训练样本时神经网络的性能，以便进行简单的比较。

<img width="300px" src="https://doc.shiyanlou.com/courses/uid1166617-20191228-1577532545948/wm">

可以看到，在 5 个世代的情况下，最好的结果是 0.9745 或 97.5％ 的准确度。这再一次打破了我们先前的纪录。

值得注意的是，如果旋转的角度过大，神经网络的性能会出现下降。由于旋转较大的角度意味着创建了实际上不能代表数字的图像，因此神经网络的性能出现了下降，这是可以理解的。想象一下，将 “3” 向一个方向旋转 90 度，这就不再是 3 了。因此，将过度旋转的图像添加到训练样本中，增加了错误样本，降低了训练的质量。对于最大化附加数据的价值，10 度看起来是最佳角度。

在 10 个世代的情况下，神经网络的性能出现了峰值，打破了记录，达到 了 0.9787 ，几乎到达 98% ！对于这种简单的神经网络而言，这是一个惊人的结果，也是最佳的一种状态。请记住，有些人会对神经网络或数据进行一些巧妙的处理，我们还未这样做，我们只是保持简单的神经网络，但是却依然取得了令人骄傲的结果。

### 实验总结

在人工智能中，神经网络是一些神奇的应用程序成功的关键部分。人们对神经网络和机器学习，特别是深度学习：也就是使用了有层次结构的机器学习方法，依然充满了巨大兴趣。在 2016 年年初，在古老的围棋对弈领域，谷歌的 DeepMind 击败了世界级大师。因此，此次事件成为了人工智能史上一个巨大的里程碑。而神经网络在此过程中也发挥了关键性作用。

<hr><div style="color: #999; font-size: 12px;"><i class="fa fa-copyright" aria-hidden="true"> 本课程内容版权归实验楼所有，禁止转载、下载及非法传播。</i></div>